In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
import os
os.chdir('/content/drive/MyDrive/GEE_Download')
print(os.getcwd())

/content/drive/MyDrive/GEE_Download


In [22]:
!ls

checkpoints	 gee_downloading.py  requirement_package_installation.txt  unet_inference_COG.py
exports2A.py	 GEEdownload.ipynb   slope.py				   utils.py
exports2A_v2.py  __pycache__	     translate_cog.py
exports2A_v3.py  remove_files.py     unet_inference_COG_argparse.py


In [23]:
!pip install -r requirement_package_installation.txt

In [24]:
import ee
import os
import random
ee.Authenticate()
ee.Initialize(project = 'tony-1122')
print(ee.String('Hello from the Earth Engine servers!').getInfo())

Hello from the Earth Engine servers!


In [25]:
%%capture
if 'google.colab' in str(get_ipython()):
  !apt-get install gdal-bin
  !pip install leafmap(raster)

!gdalinfo --version

In [26]:
from google.colab import auth
auth.authenticate_user()

In [27]:
# Path to the folder you want to create
folder_path = '/content/Clay_mangrove'

# Create the folder if it does not exist
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print("Folder created")
else:
    print("Folder already exists")

Folder already exists


In [28]:
# Base path for the folders
base_path = '/content/Clay_mangrove/Images'

# List of folder names to create
folders = ['s2', 'label', 'COG']

# Loop through the folder names and create each one
for folder in folders:
    folder_path = os.path.join(base_path, folder)
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Folder '{folder}' created")
    else:
        print(f"Folder '{folder}' already exists")

Folder 's2' already exists
Folder 'label' already exists
Folder 'COG' already exists


# List files in GCP

In [29]:
from google.cloud import storage

# Initialize a client
client = storage.Client()

# Specify the bucket name and prefix (folder)
bucket_name = 'mangrove_clay'
prefix = 'remain_sentinel_2024/'  # Folder path

# Get the bucket object
bucket = client.get_bucket(bucket_name)

# List blobs in the specified prefix (folder)
blobs = client.list_blobs(bucket_name, prefix=prefix, delimiter='/')

# Print each folder (prefix) found in the directory
folders = set()
for page in blobs.pages:
    folders.update(page.prefixes)

# Print the number of blobs in each subfolder
for folder in folders:
    subfolder_blobs = client.list_blobs(bucket_name, prefix=folder)
    blob_count = sum(1 for _ in subfolder_blobs)  # Count the blobs
    print(f'{folder}: {blob_count} files')

remain_sentinel_2024/sentinel_patches_ph_others/: 1340 files
remain_sentinel_2024/sentinel_patches_brunei/: 7 files
remain_sentinel_2024/sentinel_patches_AF_cambodia/: 4 files
remain_sentinel_2024/sentinel_patches_AF_ph/: 70 files
remain_sentinel_2024/sentinel_patches_AF_vietnam/: 55 files
remain_sentinel_2024/sentinel_patches_malaysia/: 504 files
remain_sentinel_2024/sentinel_patches_AF_myanmar/: 367 files
remain_sentinel_2024/sentinel_patches_ph/: 123 files
remain_sentinel_2024/sentinel_patches_thailand/: 276 files
remain_sentinel_2024/sentinel_patches_myanmar/: 591 files
remain_sentinel_2024/sentinel_patches_vietnam/: 5 files
remain_sentinel_2024/sentinel_patches_indo/: 1676 files
remain_sentinel_2024/sentinel_patches_AF_thailand/: 233 files


# Start downloading Images and convert to COG

---





In [39]:
country = 'brunei'
year = 2024
!python exports2A_v3.py "projects/tony-1122/assets/Mangrove_samplePoints_AF_review/mangrove_{country}_centroid_AF_review" 2024

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init
Processing 152 samples from the collection: projects/tony-1122/assets/Mangrove_samplePoints_AF_review/mangrove_brunei_centroid_AF_review for the year 2024
Input file size is 512, 512
0...10...20...30...40...50...60...70...80...90...100 - done.
Successfully converted /content/Clay_mangrove/Images/s2/00003_temp.tif to COG: /content/Clay_mangrove/Images/s2/00003.tif
Converted /content/Clay_mangrove/Images/s2/00003.tif to Cloud Optimized GeoTIFF.
s2 images: 3 (512, 512, 10)
Input file size is 512, 512
0...10...20...30...40...50...60...70...80...90...100 - done.
Successfully converted /content/Clay_mangrove/Images/s2/00005_temp.tif to COG: /content/Clay_mangrove/Images/s2/00005.tif
Converted /content/Clay_mangrove/Images/s2/00005.tif to Cloud Optimized GeoTIFF.
s2 images: 5 (512, 512, 10)
Input file size is 512, 512
0...10...20...3

# **Rename files if needed**

In [ ]:
# import os

# # Set the directory path and starting number
# directory_path = '/content/Clay_mangrove/Images/s2'
# start_number = 1775

# # Get a list of files in the directory
# files = sorted(os.listdir(directory_path))

# # Loop through the files and rename them
# for i, filename in enumerate(files):
#     file_path = os.path.join(directory_path, filename)
#     if os.path.isfile(file_path):
#         # Get the file extension
#         file_extension = os.path.splitext(filename)[1]
#         # Create the new filename
#         new_filename = f"{start_number + i}{file_extension}"
#         new_file_path = os.path.join(directory_path, new_filename)
#         # Rename the file
#         os.rename(file_path, new_file_path)

# print("Files renamed successfully.")


Files renamed successfully.


# Copy images to GCP

In [40]:
!gsutil -m cp -r /content/Clay_mangrove/Images/s2/* gs://mangrove_clay/remain_sentinel_{year}/sentinel_patches_AF_{country}/

Copying file:///content/Clay_mangrove/Images/s2/00002.tif [Content-Type=image/tiff]...
Copying file:///content/Clay_mangrove/Images/s2/00003.tif [Content-Type=image/tiff]...
Copying file:///content/Clay_mangrove/Images/s2/00004.tif [Content-Type=image/tiff]...
Copying file:///content/Clay_mangrove/Images/s2/00000.tif [Content-Type=image/tiff]...
Copying file:///content/Clay_mangrove/Images/s2/00001.tif [Content-Type=image/tiff]...
Copying file:///content/Clay_mangrove/Images/s2/00008.tif [Content-Type=image/tiff]...
Copying file:///content/Clay_mangrove/Images/s2/00009.tif [Content-Type=image/tiff]...
Copying file:///content/Clay_mangrove/Images/s2/00005.tif [Content-Type=image/tiff]...
Copying file:///content/Clay_mangrove/Images/s2/00007.tif [Content-Type=image/tiff]...
Copying file:///content/Clay_mangrove/Images/s2/00006.tif [Content-Type=image/tiff]...
Copying file:///content/Clay_mangrove/Images/s2/00010.tif [Content-Type=image/tiff]...
Copying file:///content/Clay_mangrove/Image

# Convert to Cloud Optimized GeoTIFF

In [ ]:
# !python translate_cog.py '/content/Clay_mangrove/Images/s2/' '/content/Clay_mangrove/Images/COG/'

# Remove files in folders

In [41]:
from remove_files import remove_files_in_folder
# Path to the 'label' folder
folder_path_label = '/content/Clay_mangrove/Images/label'
remove_files_in_folder(folder_path_label)

# Path to the 's2' folder
folder_path_s2 = '/content/Clay_mangrove/Images/s2'
remove_files_in_folder(folder_path_s2)

# Path to the 'COG' folder
folder_path_COG = '/content/Clay_mangrove/Images/COG'
remove_files_in_folder(folder_path_COG)

All contents of the folder /content/Clay_mangrove/Images/label have been successfully removed!!!
All contents of the folder /content/Clay_mangrove/Images/s2 have been successfully removed!!!
All contents of the folder /content/Clay_mangrove/Images/COG have been successfully removed!!!


# Visualization part

In [ ]:
# # Initialize Google Cloud Storage client
# folder = 'singapore'
# from google.cloud import storage
# client = storage.Client()
# # Specify your bucket name and the prefix (folder path in the bucket)
# bucket_name = 'mangrove_clay'
# prefix = f'sentinel_2023/sentinel_patches_{folder}/'

# bucket = client.get_bucket(bucket_name)

# # List all files in the specified prefix (folder)
# blobs = bucket.list_blobs(prefix=prefix)

# # Create a list of file paths
# gcs_paths = ['gs://' + bucket_name + '/' + blob.name for blob in blobs if blob.name.endswith('.tif')]

# # Check the number of patches found
# print(f"Found {len(gcs_paths)} patches.")

Found 40 patches.


In [ ]:
# import rasterio

# bucket = client.bucket(bucket_name)

# # Function to list files in a GCS bucket folder
# def list_gcs_files(bucket_name, prefix):
#     storage_client = storage.Client()
#     blobs = storage_client.list_blobs(bucket_name, prefix=prefix)
#     return [blob.name for blob in blobs if blob.name.endswith('.tif')]

# # Function to read and print the size of a patch (GeoTIFF file) from a downloaded file
# def print_patch_size(local_path):
#     with rasterio.open(local_path) as dataset:
#         width = dataset.width
#         height = dataset.height
#         # print(f'Patch {os.path.basename(local_path)} - Width: {width} pixels, Height: {height} pixels')
#         if width != 512 or height != 512:
#           print(f'Patch {os.path.basename(local_path)} - Width: {width} pixels, Height: {height} pixels')
#         else:
#           print('All matches with 512 x 512 pixels')

# # List all GeoTIFF files in the specified folder
# file_paths = list_gcs_files(bucket_name, prefix)

# # Download, read, and print the size of each patch
# for file_path in file_paths:
#     blob = bucket.blob(file_path)
#     # Download the file to a temporary location
#     local_path = f'/tmp/{os.path.basename(file_path)}'
#     blob.download_to_filename(local_path)

#     # Print the size of the patch
#     print_patch_size(local_path)
#     # Optionally, remove the local file after processing
#     os.remove(local_path)


All matches with 512 x 512 pixels
All matches with 512 x 512 pixels
All matches with 512 x 512 pixels
All matches with 512 x 512 pixels
All matches with 512 x 512 pixels
All matches with 512 x 512 pixels
All matches with 512 x 512 pixels
All matches with 512 x 512 pixels
All matches with 512 x 512 pixels
All matches with 512 x 512 pixels
All matches with 512 x 512 pixels
All matches with 512 x 512 pixels
All matches with 512 x 512 pixels
All matches with 512 x 512 pixels
All matches with 512 x 512 pixels
All matches with 512 x 512 pixels
All matches with 512 x 512 pixels
All matches with 512 x 512 pixels
All matches with 512 x 512 pixels
All matches with 512 x 512 pixels
All matches with 512 x 512 pixels
All matches with 512 x 512 pixels
All matches with 512 x 512 pixels
All matches with 512 x 512 pixels
All matches with 512 x 512 pixels
All matches with 512 x 512 pixels
All matches with 512 x 512 pixels
All matches with 512 x 512 pixels
All matches with 512 x 512 pixels
All matches wi

In [ ]:
# import geemap
# # Visualization parameters
# vis_params = {'bands': ['B2', 'B1', 'B0'], 'min': 400, 'max': 3600, 'gamma': 1}

# # Load the patches as Earth Engine images
# images = [ee.Image.loadGeoTIFF(gcs_path) for gcs_path in gcs_paths[0:1000]]

# # Create a mosaic of all the patches
# mosaic_image = ee.ImageCollection(images).mosaic()

# # Create a map object
# Map = geemap.Map()

# # Add the mosaic image to the map
# Map.addLayer(mosaic_image, vis_params, "Mosaic of Patches")
# Map.addLayer(grid, {'color' : 'green'}, "grid", False)
# # Display the map
# Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
# import subprocess
# import os

# # Path to your local directories
# input_directory = '/content/drive/MyDrive/Clay_mangrove/Images/s2/'
# output_directory = '/content/drive/MyDrive/Clay_mangrove/Images/COG/'

# # Create the output directory if it doesn't exist
# if not os.path.exists(output_directory):
#     os.makedirs(output_directory)

# # List all files in the input directory
# file_list = os.listdir(input_directory)

# for item in file_list:
#     # Only process .tif files
#     if item.endswith('.tif'):
#         filename = item.split('.')[0]

#         # Ensure the conversion is completed before moving on
#         print(f"Converting {filename} to COG format...")
#         subprocess.run(['gdal_translate', f'{input_directory}{filename}.tif',
#                         f'{output_directory}{filename}.tif', '-of', 'COG'], check=True)

#         print(f"Process completed for {filename}.")